# Demo Tokenizer

## BPE - Byte-Pair Encoding Tokenization

In [1]:
corpus = [
    "Ich bin ein deutscher Text.",
    "Und ich bin ein etwas längerer deutscher Text.",
    "Auch ich bin deutschsprachig.",
]

Wir verwenden den AutoTokenizer aus der HuggingFace Transformers-Library, um den Text in einzelne Zeichen zu zerlegen. Dieser AutoTokenizer ist auch bekannt als gpt2-Tokenizer

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

Während der Pre-Tokenisierung berechnen wir die Häufigkeit eines jeden Worts im Corpus.
Beachte: Ġ steht für das Leerzeichen innerhalb eines Satzes.

In [3]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'Ich': 1, 'Ġbin': 3, 'Ġein': 2, 'Ġdeutscher': 2, 'ĠText': 2, '.': 3, 'Und': 1, 'Ġich': 2, 'Ġetwas': 1, 'ĠlÃ¤ngerer': 1, 'Auch': 1, 'Ġdeutschsprachig': 1})


Jetzt erstellen wir das Grundvokabular.
Gleichzeitig erstellen wir auch unser Alphabet.
Im produktiven Kontext würden wir alle Bitfolgen unseres Zeichensatzes nehmen (ASCII, Unicode) und wir würden zusätzlich auch ein Token für 'unknown' erstellen.

In [4]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)


['.', 'A', 'I', 'T', 'U', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'n', 'p', 'r', 's', 't', 'u', 'w', 'x', '¤', 'Ã', 'Ġ']


Wir fügen ein weiteres Zeichen hinzu, um ein Textende zu kennzeichnen. So werden die drei Texte in unserem Corpus voneinander abgegrenzt.

In [5]:
vocab = ["<|endoftext|>"] + alphabet.copy()

Jetzt splitten wir jedes Wort in einzelne Zeichen.

In [7]:
splits = {word: [c for c in word] for word in word_freqs.keys()}
splits

{'Ich': ['I', 'c', 'h'],
 'Ġbin': ['Ġ', 'b', 'i', 'n'],
 'Ġein': ['Ġ', 'e', 'i', 'n'],
 'Ġdeutscher': ['Ġ', 'd', 'e', 'u', 't', 's', 'c', 'h', 'e', 'r'],
 'ĠText': ['Ġ', 'T', 'e', 'x', 't'],
 '.': ['.'],
 'Und': ['U', 'n', 'd'],
 'Ġich': ['Ġ', 'i', 'c', 'h'],
 'Ġetwas': ['Ġ', 'e', 't', 'w', 'a', 's'],
 'ĠlÃ¤ngerer': ['Ġ', 'l', 'Ã', '¤', 'n', 'g', 'e', 'r', 'e', 'r'],
 'Auch': ['A', 'u', 'c', 'h'],
 'Ġdeutschsprachig': ['Ġ',
  'd',
  'e',
  'u',
  't',
  's',
  'c',
  'h',
  's',
  'p',
  'r',
  'a',
  'c',
  'h',
  'i',
  'g']}

Die folgende Funktion berechnet die Häufigkeit eines jeden Tokenpaars 

In [8]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

Wir wenden die Funktion an und schauen das Ergebnis an: 'c' kommt am häufigsten vor.

In [9]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")


('I', 'c'): 1
('c', 'h'): 8
('Ġ', 'b'): 3
('b', 'i'): 3
('i', 'n'): 5
('Ġ', 'e'): 3
('e', 'i'): 2
('Ġ', 'd'): 3
('d', 'e'): 3
('e', 'u'): 3
('u', 't'): 3
('t', 's'): 3
('s', 'c'): 3
('h', 'e'): 2
('e', 'r'): 4
('Ġ', 'T'): 2
('T', 'e'): 2
('e', 'x'): 2
('x', 't'): 2
('U', 'n'): 1
('n', 'd'): 1
('Ġ', 'i'): 2
('i', 'c'): 2
('e', 't'): 1
('t', 'w'): 1
('w', 'a'): 1
('a', 's'): 1
('Ġ', 'l'): 1
('l', 'Ã'): 1
('Ã', '¤'): 1
('¤', 'n'): 1
('n', 'g'): 1
('g', 'e'): 1
('r', 'e'): 1
('A', 'u'): 1
('u', 'c'): 1
('h', 's'): 1
('s', 'p'): 1
('p', 'r'): 1
('r', 'a'): 1
('a', 'c'): 1
('h', 'i'): 1
('i', 'g'): 1


Wir suchen das häufigste Tokenpaar: 'ch' kommt am häufigsten vor.

In [10]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('c', 'h') 8


Wir verschmelzen 'c' und 'h' zu 'ch' und fügen das neue Token dem Vokabular hinzu.

In [14]:
merges = {("c", "h"): "ch"}
vocab.append("ch")

Die folgende Funktion wendet den gezeigten Merge-Step auf das Split-Directory an. 

In [11]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

Schauen wir das Ergebnis unseres ersten Merges an:
Aus 'I','c','h' wurde neu 'I','ch'

In [12]:
splits = merge_pair("c", "h", splits)
splits

{'Ich': ['I', 'ch'],
 'Ġbin': ['Ġ', 'b', 'i', 'n'],
 'Ġein': ['Ġ', 'e', 'i', 'n'],
 'Ġdeutscher': ['Ġ', 'd', 'e', 'u', 't', 's', 'ch', 'e', 'r'],
 'ĠText': ['Ġ', 'T', 'e', 'x', 't'],
 '.': ['.'],
 'Und': ['U', 'n', 'd'],
 'Ġich': ['Ġ', 'i', 'ch'],
 'Ġetwas': ['Ġ', 'e', 't', 'w', 'a', 's'],
 'ĠlÃ¤ngerer': ['Ġ', 'l', 'Ã', '¤', 'n', 'g', 'e', 'r', 'e', 'r'],
 'Auch': ['A', 'u', 'ch'],
 'Ġdeutschsprachig': ['Ġ',
  'd',
  'e',
  'u',
  't',
  's',
  'ch',
  's',
  'p',
  'r',
  'a',
  'ch',
  'i',
  'g']}

Wir legen eine Vokabulargröße fest und wiederholen alles bis diese Größe erreicht ist.

In [15]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [16]:
print(merges)

{('c', 'h'): 'ch', ('e', 'r'): 'er', ('Ġ', 'b'): 'Ġb', ('Ġb', 'in'): 'Ġbin', ('Ġ', 'e'): 'Ġe', ('Ġ', 'd'): 'Ġd', ('Ġd', 'e'): 'Ġde', ('Ġde', 'u'): 'Ġdeu', ('Ġdeu', 't'): 'Ġdeut', ('Ġdeut', 's'): 'Ġdeuts', ('Ġdeuts', 'ch'): 'Ġdeutsch', ('Ġe', 'in'): 'Ġein', ('Ġdeutsch', 'er'): 'Ġdeutscher', ('Ġ', 'T'): 'ĠT', ('ĠT', 'e'): 'ĠTe', ('ĠTe', 'x'): 'ĠTex', ('ĠTex', 't'): 'ĠText', ('Ġ', 'i'): 'Ġi', ('Ġi', 'ch'): 'Ġich', ('I', 'ch'): 'Ich', ('U', 'n'): 'Un', ('Un', 'd'): 'Und', ('Ġe', 't'): 'Ġet', ('Ġet', 'w'): 'Ġetw'}


In [17]:
print(vocab)

['<|endoftext|>', '.', 'A', 'I', 'T', 'U', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'n', 'p', 'r', 's', 't', 'u', 'w', 'x', '¤', 'Ã', 'Ġ', 'ch', 'er', 'Ġb', 'Ġbin', 'Ġe', 'Ġd', 'Ġde', 'Ġdeu', 'Ġdeut', 'Ġdeuts', 'Ġdeutsch', 'Ġein', 'Ġdeutscher', 'ĠT', 'ĠTe', 'ĠTex', 'ĠText', 'Ġi', 'Ġich', 'Ich', 'Un', 'Und', 'Ġet', 'Ġetw']


In [18]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [19]:
tokenize("Ich spreche deutsch")

['Ich', 'Ġ', 's', 'p', 'r', 'e', 'ch', 'e', 'Ġdeutsch']

Siehe auch: https://platform.openai.com/tokenizer 